In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import time
import os

In [2]:
tf.__version__

'2.2.0'

In [3]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 329050154285216060, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 16865402589242252223
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 9617915005333357267
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 15701463552
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 7687158442107619517
 physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"]

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
cd /content/drive/My Drive/tobigs_recommend/Phase v2. Recsys2019 Deep Modeling/GRU4REC

/content/drive/My Drive/tobigs_recommend/Phase v2. Recsys2019 Deep Modeling/GRU4REC


## Preprocessing

In [9]:
ls

checkpoint/  GRU4REC_preprocession.ipynb  test.csv
data/        GRU4REC_tf.ipynb             train.csv


In [10]:
ls

checkpoint/  GRU4REC_preprocession.ipynb  test.csv
data/        GRU4REC_tf.ipynb             train.csv


In [11]:
## set parameters

ROOT = './data/preprocessed/'
DATA_TYPE = 'full'
PATH_TO_TRAIN = ROOT + 'trivago_train_{}.txt'.format(DATA_TYPE)
PATH_TO_TEST = ROOT + 'trivago_train_{}.txt'.format(DATA_TYPE)
checkpoint_dir = './checkpoint'
if not os.path.exists(checkpoint_dir): 
    os.mkdir(checkpoint_dir)
        
layers = 1
rnn_size = 100
batch_size = 50
drop_keep_prob = 0.7

n_epochs = 3
learning_rate = 0.001
decay = 0.96
decay_steps = 1e4
grad_cap = 0
print_step = 1e3

In [12]:
## load data 
data = pd.read_csv(PATH_TO_TRAIN, sep='\t', dtype={'ItemId': np.int64})
valid = pd.read_csv(PATH_TO_TEST, sep='\t', dtype={'ItemId': np.int64})

In [13]:
## check sort data
### preprocessing에서 session, timestamp로 sorting을 하였는데,
### sorting이 중요하기 때문에 한번더 확인해본다.
### data가 클경우 sort에서 시간이 오래 걸리기 때문에 sample check을 수행한다.
def check_data_sort(dt, sample_check=False, sample_size=10000):
    if sample_check:
        sess_ids = dt['SessionId'].unique()
        sample_sess_ids = np.random.choice(sess_ids, sample_size, replace=False)
        dt = dt[np.in1d(dt.SessionId, sample_sess_ids)]
    ordered_dt = dt.sort_values(['SessionId', 'timestamp'])
    return dt.equals(ordered_dt)

print(check_data_sort(data, sample_check=True))
print(check_data_sort(valid))

True
True


In [14]:
## add item index 
### item id를 0번부터 index를 추가합니다.
itemids = data['ItemId'].unique()
n_items = len(itemids)
itemidmap = pd.Series(data=np.arange(n_items), index=itemids).to_dict()
%time data['ItemIdx'] = data['ItemId'].map(lambda x: itemidmap[x])
data[:5] 

CPU times: user 2.48 s, sys: 109 ms, total: 2.59 s
Wall time: 2.59 s


,SessionId,ItemId,timestamp,ItemIdx
0,2,445081,1.541070e+09,0
1,2,445081,1.541070e+09,0
2,2,445081,1.541070e+09,0
3,2,445081,1.541070e+09,0
4,2,445081,1.541070e+09,0


In [15]:
## offset sessions
### 각 세션의 시작점의 index list를 만든다.
### 즉, 첫번째 SessionId 2의 시작점은 0이고 
offset_sessions = np.zeros(data['SessionId'].nunique()+1, dtype=np.int32)
offset_sessions[1:] = data.groupby('SessionId').size().cumsum()
offset_sessions[:5] 

array([ 0, 51, 56, 59, 64], dtype=int32)

In [16]:
data.groupby('SessionId').size().cumsum()

SessionId
2              51
3              56
9              59
10             64
11            342
           ...   
412714    5900160
412715    5900165
412717    5900190
412720    5900200
412722    5900211
Length: 225223, dtype: int64

## Prepare Model

In [17]:
## placeholder & learning rate
tf.compat.v1.disable_eager_execution()

X = tf.compat.v1.placeholder(tf.int32, [batch_size], name='input')
Y = tf.compat.v1.placeholder(tf.int32, [batch_size], name='output')
States = [tf.compat.v1.placeholder(tf.float32, [batch_size, rnn_size], name='rnn_state') for _ in range(layers)]
global_step = tf.Variable(0, name='global_step', trainable=False)
lr = tf.maximum(1e-5,tf.compat.v1.train.exponential_decay(
    learning_rate, global_step, decay_steps, decay, staircase=True))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [18]:
## gru weigths
### input item에 대한 embedding matrix 와
### next item 즉 output을 위한 softmax W, b matrix를 구성한다.
with tf.compat.v1.variable_scope('gru_layer', reuse=tf.compat.v1.AUTO_REUSE):
    #sigma = sigma if sigma != 0 else np.sqrt(6.0 / (n_items + rnn_size))
    #initializer = tf.random_uniform_initializer(minval=-sigma, maxval=sigma)
    initializer = tf.compat.v1.glorot_uniform_initializer()
    embedding = tf.compat.v1.get_variable('embedding', [n_items, rnn_size], initializer=initializer)
    softmax_W = tf.compat.v1.get_variable('softmax_w', [n_items, rnn_size], initializer=initializer)
    softmax_b = tf.compat.v1.get_variable('softmax_b', [n_items], initializer=tf.zeros_initializer())

In [19]:
## gru_cell
### ㅁt => ㅁt+1 => ㅁt+2 => ... 
### 위와 같은 recurrent network에서 ㅁ. 즉, 단일 gru cell을 말한다.
with tf.compat.v1.variable_scope('gru_cell', reuse=tf.compat.v1.AUTO_REUSE):
    cell = tf.compat.v1.nn.rnn_cell.GRUCell(rnn_size, activation=tf.nn.tanh)
    drop_cell = tf.compat.v1.nn.rnn_cell.DropoutWrapper(cell, output_keep_prob=drop_keep_prob)
    stacked_cell = tf.compat.v1.nn.rnn_cell.MultiRNNCell([drop_cell] * layers)

Instructions for updating:
This class is equivalent as tf.keras.layers.GRUCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.


In [20]:
## feedforward gur_cell
### 예를들어 seesion 1의 item sequence가 5, 7, 9 라면,
### 첫번째 배치에서 아이템 5에 대하여 embedding 하여 inputs를 추출하고,
### session 1의 초기 states로 부터 output과 final_state를 계산한다.
### 두번째 배치에서는 아이템 7에 대한 inputs이 들어가고 아이템 5로 부터 계산된
### final state로 아이템 7에 대한 output과 final_state가 다시 계산된다.
### 즉, 배치 순서로 각 seesion의 item sequence가 recurrent하게 학습되는 것이다.
inputs = tf.nn.embedding_lookup(embedding, X)
output, state_ = stacked_cell(inputs, tuple(States))
final_state = state_

Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [21]:
## calculate cost(loss)
### 학습일 경우 negative sampling을 통해 
### cross-entropy loss로 계산하였다. 

### for training
sampled_W = tf.nn.embedding_lookup(softmax_W, Y)
sampled_b = tf.nn.embedding_lookup(softmax_b, Y)
logits = tf.matmul(output, sampled_W, transpose_b=True) + sampled_b
### cross-entropy loss
yhat = tf.nn.softmax(logits)
cost = tf.reduce_mean(-tf.compat.v1.log(tf.compat.v1.diag_part(yhat)+1e-24))

### for prediction
logits_all = tf.matmul(output, softmax_W, transpose_b=True) + softmax_b
## logits_all
yhat_all = tf.nn.softmax(logits_all)

In [22]:
## optimize
### Adam optimizer를 사용한다.
optimizer = tf.compat.v1.train.AdamOptimizer(lr)
### grad_cap>0 다면, minimize시 gradient cliping을 수행한다.
### gradient cliping을 수행하는 이유는 다음 블로그 참조 (https://dhhwang89.tistory.com/90)
### 간략하게 학습 중에 gradient가 급격하게 변하는 지점이 발생할 수 있는데, 이는 기존 minima를 찾아가는 방향이 
### 급변할 수 있기 때문에, 이를 방지하기 위해 수행한다.
### 본 학습에서는 cliping을 하지 않는데, 유사하게 learning rate decay을 사용하기 때문인 것으로 생각됨.
tvars = tf.compat.v1.trainable_variables()
gvs = optimizer.compute_gradients(cost, tvars)
if grad_cap > 0:
    capped_gvs = [(tf.clip_by_norm(grad, grad_cap), var) for grad, var in gvs]
else:
    capped_gvs = gvs 
train_op = optimizer.apply_gradients(capped_gvs, global_step=global_step)

## Training

In [20]:
## session start
sess = tf.compat.v1.Session()
sess.run(tf.compat.v1.global_variables_initializer())
saver = tf.compat.v1.train.Saver(tf.compat.v1.global_variables())

### understanding data feed

In [21]:
### 1. 초기 세팅으로 batch_size 만큼 index array를 만들고 maxiter값을 저장한다.
### 2. start는 offset_session(sessionid의 시작 index) 에서 iters를 추출한다.
###    즉, 첫 50개 sessionid의 시작 index를 추출다.
### 3. end는 각 세션에서 다음 세션의 시작되는 index를 추출한다.
iters = np.arange(batch_size)
maxiter = iters.max() 
print(iters[:5], "...", iters[-5:])
print(maxiter)
start = offset_sessions[iters]
end = offset_sessions[iters+1]
print(start[:5])
print(end[:5]) 

[0 1 2 3 4] ... [45 46 47 48 49]
49
[ 0 51 56 59 64]
[ 51  56  59  64 342]


In [22]:
### 1. end - start의 최소 값을 추출한다.
### 만약 최소값이 3라면 즉, 하나의 세션의 item이 두개라면,
### 첫번째 item은 input으로 사용되고 두번째 item은 output으로 사용된다.
### 그리고 다음 배체에서 두번째 item은 input으로 사용되고 세번째 item이 output으로 사용된다.
### 해당 세션은 배치가 두번 돈 후 더이상 학습 할 수 없으므로 다음 세션으로 교체되어야 한다.
### 만약 최소값으 2라면, 해당 세션은 1번 배치 후 다음 세션으로 교체되어야 한다.
### 즉, end - start - 1의 최소 값은 현재 배치된 session의 반복 수를 의미한다.
### 2. out_idx는 각 session의 첫 itemidx를 나타낸다.
minlen = (end-start).min()
out_idx = data.ItemIdx.values[start]
print(minlen) 
print(out_idx[:5]) 

2
[ 0  3  5  7 11]


In [23]:
### 위에서 설명한 것과 같이, 각 세션의 첫번째 아이템이 in, 두번째 아이템이 out이 된 후 학습에 사용된다.
### minlen - 1 의 수만큼 반복(i)되어 학습한다.
i = 0
in_idx = out_idx
out_idx = data.ItemIdx.values[start+i+1]
print(in_idx[:5])
print(out_idx[:5])

[ 0  3  5  7 11]
[ 0  3  6  7 11]


In [24]:
### 1. 위에서 minlen - 1만큼 반복이 완료 되었다는 것은, 교체되어야 할 sessionid가 있다는 말이다.
### start 즉, 세션의 시작 index에서 minlen-1 만큼 증가(반복) 후에 end-start가 1보다 작다는 것은
### 해당 세션의 item이 모두 사용되었다는 것이다. 
### 2. 해당 index를 mask에 저장한다.
### 즉, 배치 index 0(2-1), 2(10-9),... 에 위치한 session의 item은 모두 사용된 것이다.
start = start+minlen-1
mask = np.arange(len(iters))[(end-start)<=1]
print(end[:5])
print(start[:5])
print(mask[:5])

[ 51  56  59  64 342]
[ 1 52 57 60 65]
[ 9 13 17 19 25]


In [25]:
### 1. iters 즉, 배치 index의 0번에 maxiter(49) + 1인 다음 session index로 교체한다.
### 마찬가지고 배치 index 2번에는 그다음 session index 51로 교체한다.
iters[0] = 50
iters[2] = 51
print(iters[:10])
### 2. start와 end는 각 데이터에서 각 세션id의 시작 index와 다음 세션의 시작 index 의미하였다.
### 교체된 session 지점에 새로운 세션id에 대한 시작 index와 다음 세션의 시작 index로 교체한다.
start[0] = offset_sessions[50]
end[0] = offset_sessions[50+1]
start[2] = offset_sessions[51]
end[2] = offset_sessions[51+1]
print(start[:10])
print(end[:10])

[50  1 51  3  4  5  6  7  8  9]
[1398   52 1466   60   65  343  346  349  352  381]
[1466   56 1471   64  342  345  348  351  380  382]


In [26]:
### 위 두 단락에서 session item이 다 사용된 iter 0, 2번 index에 대해서만 교체하였는데,
### 모든 교체되어야할 index 즉, 모든 mask에 대하여 session 교체를 수행한다.
for idx in mask:
    maxiter += 1
    if maxiter >= len(offset_sessions)-1:
        finished = True
        break
    iters[idx] = maxiter
    start[idx] = offset_sessions[maxiter]
    end[idx] = offset_sessions[maxiter+1]
print(start[:10])
print(end[:10]) 

[1398   52 1466   60   65  343  346  349  352 1398]
[1466   56 1471   64  342  345  348  351  380 1466]


In [27]:
### 전체 과정 (한번의 epoch)을 거치면, 마지막 sessionid 의 index 5900200에서 종료된다.
finished = False
endpoint_count = 0
while not finished:
    minlen = (end-start).min()
    out_idx = data.ItemIdx.values[start]
    for i in range(minlen-1):
        in_idx = out_idx
        out_idx = data.ItemIdx.values[start+i+1]
        endpoint_count += len(out_idx)
    
    start = start+minlen-1
    mask = np.arange(len(iters))[(end-start)<=1]

    for idx in mask:
        maxiter += 1
        if maxiter >= len(offset_sessions)-1:
            finished = True
            break
        iters[idx] = maxiter
        start[idx] = offset_sessions[maxiter]
        end[idx] = offset_sessions[maxiter+1]
        
print(max(start))
data[-5:]

5900200


,SessionId,ItemId,timestamp,ItemIdx
5900206,412722,104632,1.541245e+09,11161
5900207,412722,104632,1.541245e+09,11161
5900208,412722,104632,1.541245e+09,11161
5900209,412722,1244689,1.541245e+09,12155
5900210,412722,1244990,1.541245e+09,8556


In [28]:
### SessionId 별 size -1의 합의 전체 학습할 수 있는 데이터인데,
### 위 data feed logic에서는 그보다 적게 끝난다.
### batch size 만큼 session placeholder를 만들고 session의 item이 다 소진되면,
### 다름 세션으로 교체하는 방식인데, 마지막 세션이 교체된 후 minlen 까지만 학습하게 된다.
### 예를들어 마지막 50개 세션에서 49개가 item이 3개가 있고 1개가 2개의 item을 가지고 있다면,
### minlen은 1이 되어 한번 학습 후, 나머지 49개의 세션에는 아직 학습할 item이 남아있지만, 
### 한개의 빈자리에 더이상 교체될 세션이 없어지므로 학습을 종료하게 되기 때문에 enpoint count가 적게 나타난다.
### 예측에서는 위와 조금 다른 방식으로 logic을 바꿔 모든 데이터를 feed할 것이다.
### (학습에서도 모든 데이터를 feed하는 방식으로 바꾸어도 무방할 것으로 보인다.)

print(endpoint_count)
print(sum(data.groupby('SessionId').size() - 1))

5671300
5674988


## training


In [29]:
### 위 data feeding에서 in_idx, out_idx 후에 실제 학습을 수행하여, 
### epoch만큼 학습을 진행한다.
with tf.device('/GPU:0'):

  tic = time.time()
  for epoch in range(n_epochs):
      epoch_cost = []
      state = [np.zeros([batch_size, rnn_size], dtype=np.float32) for _ in range(layers)]
      iters = np.arange(batch_size)
      maxiter = iters.max()
      
      start = offset_sessions[iters]
      end = offset_sessions[iters+1]
      
      finished = False
      while not finished:
          minlen = (end-start).min()
          out_idx = data.ItemIdx.values[start]
          for i in range(minlen-1):
              in_idx = out_idx
              out_idx = data.ItemIdx.values[start+i+1]
              # prepare inputs, targeted outputs and hidden states
              fetches = [cost, final_state, global_step, lr, train_op]
              feed_dict = {X: in_idx, Y: out_idx}
              for j in range(layers): 
                  feed_dict[States[j]] = state[j]
              
              cost_, state, step, lr_, _ = sess.run(fetches, feed_dict)
              epoch_cost.append(cost_)
                  
              if step == 1 or step % print_step == 0:
                  avgc = np.mean(epoch_cost)
                  print('Epoch {}\tStep {}\tlr: {:.5f}\tloss: {:.4f}\tElapsed: {:.1f}'.
                        format(epoch, step, lr_, avgc, time.time()-tic))

          start = start+minlen-1
          mask = np.arange(len(iters))[(end-start)<=1]
          for idx in mask:
              maxiter += 1
              if maxiter >= len(offset_sessions)-1:
                  finished = True
                  break
              iters[idx] = maxiter
              start[idx] = offset_sessions[maxiter]
              end[idx] = offset_sessions[maxiter+1]
          if len(mask):
              for i in range(layers):
                  state[i][mask] = 0
          
      avgc = np.mean(epoch_cost)
      if np.isnan(avgc):
          print('Epoch {}: Nan error!'.format(epoch, avgc))
          break
      saver.save(sess, '{}/gru-model'.format(checkpoint_dir), global_step=epoch)
print("1 epoch elapsed time:", time.time() - tic)

Epoch 0	Step 0	lr: 0.00100	loss: 3.9120	Elapsed: 1.6
Epoch 0	Step 1	lr: 0.00100	loss: 3.9120	Elapsed: 1.7
Epoch 0	Step 1000	lr: 0.00100	loss: 2.8780	Elapsed: 9.6
Epoch 0	Step 2000	lr: 0.00100	loss: 2.6208	Elapsed: 17.5
Epoch 0	Step 3000	lr: 0.00100	loss: 2.5079	Elapsed: 25.5
Epoch 0	Step 4000	lr: 0.00100	loss: 2.4354	Elapsed: 33.4
Epoch 0	Step 5000	lr: 0.00100	loss: 2.3686	Elapsed: 41.3
Epoch 0	Step 6000	lr: 0.00100	loss: 2.3171	Elapsed: 49.3
Epoch 0	Step 7000	lr: 0.00100	loss: 2.2622	Elapsed: 57.2
Epoch 0	Step 8000	lr: 0.00100	loss: 2.2238	Elapsed: 65.1
Epoch 0	Step 9000	lr: 0.00100	loss: 2.1838	Elapsed: 73.0
Epoch 0	Step 10000	lr: 0.00096	loss: 2.1463	Elapsed: 81.0
Epoch 0	Step 11000	lr: 0.00096	loss: 2.1172	Elapsed: 88.9
Epoch 0	Step 12000	lr: 0.00096	loss: 2.0852	Elapsed: 96.8
Epoch 0	Step 13000	lr: 0.00096	loss: 2.0578	Elapsed: 104.7
Epoch 0	Step 14000	lr: 0.00096	loss: 2.0316	Elapsed: 112.7
Epoch 0	Step 15000	lr: 0.00096	loss: 2.0025	Elapsed: 120.7
Epoch 0	Step 16000	lr: 0.00096	

In [30]:
sess.close()

### Prediction & Evaluation
valid(test) 데이터에 대하여 예측과 평가를 수행한다.  

*evaluation metric*    
  
- Recall@20  
   예측한 top 20 아이템 중에 정답 아이템이 있는지 1 or 0으로 평가 후 전체를 평균함  
- MRR@20 (mean reciprocal rank)  
   정답 아이템의 rank의 역수를 취한 후 전체를 평균함  
   $mrr = \frac{1}{N} \sum\limits_{i=1}^{N} {\frac{1}{rank_{i}}}$
   rank가 20위 안에 들지 않으면 0으로 처리한다.  

In [23]:
## parameters
cut_off = 25     # @20
batch_size = 50

In [24]:
graph = tf.Graph()
with graph.as_default():
    with tf.Session as sess:
        tf.saved_model.loader.load(
            sess,
            [tag_constants.SERVING],
        'path/to/your/location/',
        )
        batch_size_placeholder = graph.get_tensor_by_name('batch_size_placeholder:0')
        features_placeholder = graph.get_tensor_by_name('features_placeholder:0')
        labels_placeholder = graph.get_tensor_by_name('labels_placeholder:0')
        prediction = graph.get_tensor_by_name('dense/BiasAdd:0')

        sess.run(prediction, feed_dict={
            batch_size_placeholder: some_value,
            features_placeholder: some_other_value,
            labels_placeholder: another_value})

AttributeError: ignored

In [25]:
## session restore
### 마지막(최신) 학습 checkpoint 정보를 restore한다.
sess = tf.compat.v1.Session()
saver = tf.compat.v1.train.Saver(tf.compat.v1.global_variables())
ckpt = tf.compat.v1.train.latest_checkpoint(checkpoint_dir)
saver.restore(sess, ckpt)

INFO:tensorflow:Restoring parameters from ./checkpoint/gru-model-2


In [26]:
## valdation data set
valid['ItemIdx'] = valid['ItemId'].map(lambda x: itemidmap[x])
valid[:5]

,SessionId,ItemId,timestamp,ItemIdx
0,2,445081,1.541070e+09,0
1,2,445081,1.541070e+09,0
2,2,445081,1.541070e+09,0
3,2,445081,1.541070e+09,0
4,2,445081,1.541070e+09,0


In [27]:
## valid offset sessions
### 위 학습과 동일하게 각 세션의 시작점의 index list를 만든다.
offset_sessions = np.zeros(valid['SessionId'].nunique()+1, dtype=np.int32)
offset_sessions[1:] = valid.groupby('SessionId').size().cumsum()
offset_sessions[:5]

array([ 0, 51, 56, 59, 64], dtype=int32)

In [28]:
## init prediction
### 예측 세션의 배치 사이즈 보다 작을 경우 배치 사이즈를 조정한다.
if len(offset_sessions) - 1 < batch_size:
    batch_size = len(offset_sessions) - 1
    
### training step과 동일
iters = np.arange(batch_size).astype(np.int32)
maxiter = iters.max()
start = offset_sessions[iters]
end = offset_sessions[iters+1]
in_idx = np.zeros(batch_size, dtype=np.int32)
predict_state = [np.zeros([batch_size, rnn_size], dtype=np.float32) for _ in range(layers)]

In [29]:
predict_state

[array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)]

## prediction & evaluation

In [30]:
### data feeding 요약
### 학습과는 조금 다르게 valid_mask를 설정하여, batch placeholder에 더이상 feed할 세션이 없어지면,
### 해당 위치를 꺼가는 방식으로 모든 batch placeholder가 없어질 때까지 feed하는 것이다.
evalutation_point_count = 0
mrr, recall = 0.0, 0.0
tic = time.time() 
while True:
    ### iters는 batch placeholder로 0보다 큰 즉, 마지막 세션까지는 모든 위치를 켜두고
    ### 아래에서 session 데이터가 다 소진되면 해당 위치를 -1로 할당할 것이다.
    ### valid_mask가 0이 되면 즉 모든 위치가 꺼지면 학습을 종료한다.
    valid_mask = iters >= 0
    if valid_mask.sum() == 0:
        print("break at endpoint", evalutation_point_count)
        break
        
    start_valid = start[valid_mask]
    minlen = (end[valid_mask]-start_valid).min()
    in_idx[valid_mask] = valid.ItemIdx.values[start_valid]
    
    for i in range(minlen-1):
        out_idx = valid.ItemIdx.values[start_valid+i+1]
        ## --- prediction --- ##
        fetches = [yhat_all, final_state]
        feed_dict = {X: in_idx}
        for j in range(layers): 
            feed_dict[States[j]] = predict_state[j]
        preds, predict_state = sess.run(fetches, feed_dict)
        preds = pd.DataFrame(data=np.asarray(preds).T)
        preds.fillna(0, inplace=True) ### preds shape: (item_size, batch_size)
        print('-----------preds-------')
        print('preds {} {}'.format(i, preds))
        
        ## --- evaluation --- ##
        in_idx[valid_mask] = out_idx
        ### 정답 아이템 prediction 값보다 높은 아이템이 몇개인지 카운트 하여 rank를 계산한다.
        ranks = (preds.values.T[valid_mask].T > 
                 np.diag(preds.loc[in_idx].values)[valid_mask]).sum(axis=0) + 1
        ### cutoff에 따른 recall과 mrr을 계산한다.
        rank_ok = ranks < cut_off
        recall += rank_ok.sum()
        mrr += (1.0 / ranks[rank_ok]).sum()
        evalutation_point_count += len(ranks)
        
    start = start+minlen-1
    mask = np.arange(len(iters))[(valid_mask) & (end-start<=1)]
    
    for idx in mask:
        maxiter += 1
        ## 더 이상 할당할 세션이 없으면 해당 위치에 -1을 할당하여 끈다.
        if maxiter >= len(offset_sessions)-1:
            iters[idx] = -1
        else:
            iters[idx] = maxiter
            start[idx] = offset_sessions[maxiter]
            end[idx] = offset_sessions[maxiter+1]
            
    if len(mask):
        for i in range(layers):
            predict_state[i][mask] = 0

### 최종 matric을 계산함.
recall = recall/evalutation_point_count
mrr = mrr/evalutation_point_count
print("recall: ", recall, "mrr:", mrr, "elapsed time:", time.time()-tic)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
128579  3.307087e-06  1.358057e-07  ...  1.687418e-07  3.273369e-06

[128580 rows x 50 columns]
-----------preds-------
preds 0                   0             1   ...            48            49
0       4.771339e-07  4.599954e-07  ...  1.419495e-07  1.715170e-06
1       1.715325e-07  1.769107e-07  ...  1.321670e-07  2.424947e-06
2       3.360266e-07  3.014377e-07  ...  6.461463e-08  1.897015e-06
3       2.083653e-09  1.249737e-06  ...  1.881590e-09  6.120985e-07
4       3.277525e-08  3.014565e-06  ...  2.851516e-09  8.357806e-07
...              ...           ...  ...           ...           ...
128575  1.994273e-06  8.091346e-07  ...  1.377924e-07  1.674791e-06
128576  6.992571e-07  8.030461e-07  ...  3.834772e-08  1.352635e-06
128577  7.200400e-07  6.920320e-07  ...  3.669012e-08  1.366477e-06
128578  6.858879e-07  7.632059e-07  ...  3.537471e-08  1.411254e-06
128579  1.519729e-06  5.836471e-07  ...  5.075997e-08  1.390684e-06

[128580 rows x 50 c

KeyboardInterrupt: ignored